In [2]:
!pip install transformers fugashi ipadic pytorch-lightning

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
     |████████████████████████████████| 490 kB 45.1 MB/s 
     |████████████████████████████████| 13.4 MB 216 kB/s 
     |████████████████████████████████| 925 kB 38.4 MB/s 
     |████████████████████████████████| 596 kB 36.7 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 282 kB 41.5 MB/s 
     |████████████████████████████████| 125 kB 47.6 MB/s 
     |████████████████████████████████| 829 kB 35.5 MB/s 
     |████████████████████████████████| 1.3 MB 43.5 MB/s 
     |████████████████████████████████| 271 kB 44.3 MB/s 
     |████████████████████████████████| 160 kB 32.0 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=6364264e60f0775754d7e15d8b22eb555927c4e1b54685318c01c332765c4dd0
  Stored in directory: /root/.cache/pi

In [3]:
from transformers import RobertaForSequenceClassification, T5Tokenizer

In [9]:
import os
from collections import Counter

# default
import pandas as pd
import numpy as np

# 前処理
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# モデル
import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# チューニング
# optuna

# 評価
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [10]:
# 定数
DIR_PATH = "drive/MyDrive/Colab Notebooks/datasets/livedoor-news"
FILE_PATH = os.path.join(DIR_PATH, "data.csv")
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
CONTENT_MAX_LEN = 128
RANDOM_STATE = 42

In [25]:
bert_tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=10)

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

In [26]:
# bert_tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
# # tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

# model = RobertaForSequenceClassification.from_pretrained("rinna/japanese-roberta-base")

In [27]:
df = pd.read_csv(FILE_PATH)
df = df[["content", "content_for_ml", "content_for_nn", "category"]]
df.head()

,content,content_for_ml,content_for_nn,category
0,世界各国で公開され、全米を始め各国で記録ラッシュが続いている映画『アベンジャーズ』の第2弾と...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,movie-enter
1,日本中に大ブームを巻き起こした名作「ひみつのアッコちゃん」の実写映画化。アッコちゃん役の綾瀬...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッコ ちゃん 」 の...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッ ##コ ちゃん ...,movie-enter
2,全米では有料ケーブルテレビでのオンエアにも関わらずドラマ歴代最高視聴率を記録。すでに一部ファ...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,movie-enter
3,今年9月に全米3大ネットワークの1つ「CBS」で放送開始と共に、瞬く間に「THEEVENT/...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,movie-enter
4,ペ・ドゥナとソ・ジソクの主演で贈る、遅咲きの歌姫が恋と夢のステージで輝く人生逆転シンデレラ・...,ペ・ドゥナ と ソ・ジソク の 主演 で 贈る 、 遅咲き の 歌姫 が 恋 と 夢 の ス...,ペ ##・ ##ドゥ ##ナ と ソ ##・ ##ジ ##ソ ##ク の 主演 で 贈る 、...,movie-enter


# 前処理

In [28]:
cat_le = LabelEncoder()
df["category_id"] = cat_le.fit_transform(df.category)
print({k:v for k, v in enumerate(cat_le.classes_)})
df.head()

{0: 'dokujo-tsushin', 1: 'it-life-hack', 2: 'kaden-channel', 3: 'livedoor-homme', 4: 'movie-enter', 5: 'peachy', 6: 'smax', 7: 'sports-watch', 8: 'topic-news'}


,content,content_for_ml,content_for_nn,category,category_id
0,世界各国で公開され、全米を始め各国で記録ラッシュが続いている映画『アベンジャーズ』の第2弾と...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,movie-enter,4
1,日本中に大ブームを巻き起こした名作「ひみつのアッコちゃん」の実写映画化。アッコちゃん役の綾瀬...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッコ ちゃん 」 の...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッ ##コ ちゃん ...,movie-enter,4
2,全米では有料ケーブルテレビでのオンエアにも関わらずドラマ歴代最高視聴率を記録。すでに一部ファ...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,movie-enter,4
3,今年9月に全米3大ネットワークの1つ「CBS」で放送開始と共に、瞬く間に「THEEVENT/...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,movie-enter,4
4,ペ・ドゥナとソ・ジソクの主演で贈る、遅咲きの歌姫が恋と夢のステージで輝く人生逆転シンデレラ・...,ペ・ドゥナ と ソ・ジソク の 主演 で 贈る 、 遅咲き の 歌姫 が 恋 と 夢 の ス...,ペ ##・ ##ドゥ ##ナ と ソ ##・ ##ジ ##ソ ##ク の 主演 で 贈る 、...,movie-enter,4


In [29]:
X_train, X_test, y_train, y_test = train_test_split(df.content, df.category_id, test_size=0.2, random_state=RANDOM_STATE, stratify=df.category_id)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train)
X_train.shape, X_train.shape, X_val.shape, y_val.shape, X_test.shape, X_test.shape

((4714,), (4714,), (1179,), (1179,), (1474,), (1474,))

In [30]:
X_test.head()

7361    vol.3:谷中・根津・千駄木の「谷根千」下町地域Camera;ROLLEICORDIV、R...
7258    転職者なら誰でも気になる採用する側の心理。しかし、実際に人事担当者から本音を聞きだすことはな...
5614    「もともと子供嫌いで、友達とかが子供を連れてきても、どう接していいのか全然わからなかったんで...
4198    今年は、女子サッカーワールドカップでの「なでしこジャパン」優勝や、バレーボール世界選手権での...
2174    7日、NEWSポストセブンが配信した「ブラック企業OL内定式で「出社時はPC持参で」と言われ...
Name: content, dtype: object

In [31]:
def create_datasets(x_tr, y_tr):
  dataset_for_loader = []
  for text, cat_id in zip(x_tr, y_tr):
    encoding = bert_tokenizer(
        text,
        max_length=CONTENT_MAX_LEN,
        padding="max_length",
        truncation=True
    )
    encoding["labels"] = cat_id
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
  return dataset_for_loader

In [32]:
dataset_for_loader_train = create_datasets(X_train, y_train)
dataset_for_loader_val = create_datasets(X_val, y_val)
dataset_for_loader_test = create_datasets(X_test, y_test)

# 学習


In [33]:
dataloader_train = DataLoader(
    dataset_for_loader_train, batch_size=32, shuffle=True)

dataloader_val = DataLoader(dataset_for_loader_val, batch_size=256)
dataloader_test = DataLoader(dataset_for_loader_test, batch_size=256)

In [34]:
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            MODEL_NAME,
            num_labels=num_labels
        )
        
    def forward(self, batch):
      output = self.bert_sc(**batch)
      return output


    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [35]:
# 6-15
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [36]:
# 6-16
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [37]:
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:679: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.8805969953536987}
--------------------------------------------------------------------------------
Accuracy: 0.88


In [38]:
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /content/model/epoch=4-step=739.ckpt
ベストモデルの検証データに対する損失:  tensor(0.3408, device='cuda:0')


In [39]:
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [40]:
test_pred = trainer.predict(model=model, dataloaders=dataloader_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 148it [00:00, ?it/s]

In [41]:
test_pred_labels = []
for d in test_pred:
  test_pred_labels += d.logits.argmax(-1).tolist()

In [42]:
print("test_f1_macro", f1_score(y_test, test_pred_labels, average="macro"))


test_f1_macro 0.8728001258010543


In [ ]:
confusion_matrix(y_test, test_pred_labels)

array([[148,   2,   2,   3,   4,  14,   0,   1,   0],
       [  3, 158,   6,   6,   0,   0,   1,   0,   0],
       [  4,  12, 144,   2,   3,   2,   2,   1,   3],
       [  6,   8,   7,  70,   5,   0,   2,   3,   1],
       [  3,   0,   2,   1, 161,   4,   0,   0,   3],
       [  8,   0,   0,   3,   7, 150,   1,   0,   0],
       [  0,   0,   3,   0,   0,   2, 169,   0,   0],
       [  0,   0,   0,   1,   0,   0,   0, 178,   1],
       [  2,   3,   0,   0,   0,   0,   0,   4, 145]])

### ナイーブベイズ

/bin/bash: DIR_PATH: command not found
drive/  sample_data/
